In [1]:
import sys
import os
import time
from datetime import timedelta, datetime, tzinfo
import numpy as np
import matplotlib.pyplot as plt
import h5py
import pickle

from kafka import KafkaConsumer, TopicPartition

from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils, OffsetRange

from SharpWriter import SharpWriter
from SparkSharpRunner import SparkSharpRunner

# make graphics inline
%matplotlib inline

# Creating the Kafka consumer

In [2]:
topic='topic-a'

partition = TopicPartition(topic, 0)

consumer = KafkaConsumer(
    group_id='my-group',
    bootstrap_servers=['localhost:9092'],
    auto_offset_reset='latest',
    enable_auto_commit=False)

consumer.assign([partition])

# Getting the scan data from the Kafka topic

Select the initial offset

In [3]:
# consumer.seek(partition, 9009)
start_offset = consumer.position(partition)
print(start_offset)

6


Get the number of messages of the scan data

In [4]:
msg = next(consumer)
cxifile = pickle.loads(msg.value)
print("%s:%d:%d: n:%s" % (msg.topic, msg.partition, msg.offset, cxifile) ) 

topic-a:0:6: n:../../files/hxn26.cxi


# Running the SHARP-NSLS2 MPI/GPU application on four workers

In [5]:
partitions = 1

args = ['sharp-nsls2', '-o', '10',  cxifile]
sharpRunner = SparkSharpRunner()

print("running sharp-mpi on spark workers...");
t1 = datetime.now();
tsharp = sharpRunner.run_with_spark(args, partitions)
t2 = datetime.now();
print ("total processing time: ", (t2 - t1))

print ("sharp time on each worker: ")
for i in range(0, partitions):
    print(i, "initialization + reconstruction time: ", tsharp[i])

running sharp-mpi on spark workers...
total processing time:  0:00:23.156416
sharp time on each worker: 
0 initialization + reconstruction time:  0:00:16.860278
